## 01MAR - Actividad Whitepapers

## Artículo 01 

### The NumPy array: a structure for efficient numerical computation

#### Actividad 01.01 - Ampliar dicha explicación, aportando posibles restricciones o limitaciones a dicho sistema y ejemplos propios de los casos de uso.

Como ya se ha explicado en el el artículo, el mecanismo de Broadcasting es utilizado en la librería Numpy para hacer operaciones como la suma, resta y multiplicación de arrays de manera más óptima. Una breve explicación de lo que hace este mecanismo, es el de igualar las columnas y renglones de los arrays involucrados para así poder realizar las operaciones matemáticas. Veamos ahora cómo es que iguala ambos arreglos.

Supongamos que tenemos los siguientes arrays:

$$ A = \begin{pmatrix} 0 & 1 & 2 \\ 0 & 1 & 2 \\ 0 & 1 & 2 \end{pmatrix}$$

$$ B = \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}$$

y queremos realizar la operación $A + B$, pues bien, con el mecanismo de Broadcasting expandimos el Array B para que sea igual al arreglo B y así poder realizar la operación, sin embargo, antes de proceder a realizar el Broadcasting, es necesario mencionar que existen tres reglas:

**1.** Si los dos arrays difieren en sus dimensiones, el arreglo con una dimensión menor será llenado con unos de su lado izquierdo.\
**2.** Si el shape de ambos arrays difiere en cualquier dimensión, el array que tenga un shape de 1 será agrandado para igualar el shape del otro array.\
**3.** Si en cualquier dimensión el tamaño de los arrays difiere, y este tamaño es diferente a 1, será un error.

Ahora, retomando el ejemplo anterior, veamos cómo se aplican estas reglas.

**Shape del array A y B:**

- A.shape = (3,3)
- B.shape = (3,)

**Regla 1: El array con una dimensión menor será llenado con 1´s a la izquierda:**

- A.shape = (3,3)
- B.shape = (1,3)

**Regla 2: El array que tenga un shape diferente y que sea igual a 1 será agrandado para igualar los shapes de ambos arrays:**

- A.shape = (3,3)
- B.shape = (3,3)

$$A + B = \begin{pmatrix} 1 & 2 & 3 \\ 1 & 2 & 3 \\ 1 & 2 & 3 \end{pmatrix}$$

Es importante notar que habrá ocasiones donde será necesario expandir el array por la derecha y no por la izquierda, por ejemplo, tenemos los dos siguientes arrays:

- A.shape = (3, 2)
- B.shape = (3, )

En este caso sería útil rellenar el shape de B con 1´s a la derecha, sin embargo esto podría causar ambigüedades además de que iría en contra la regla de solamente rellenar hacia la izquierda.

#### Actividad 01.02 - Verificar la eficacia y mejora posible de rendimiento del uso de dicha técnica sobre ndarrays de tamaños grandes.

Para entender por qué el uso del memory mapping resulta un beneficio en la mayoría de las veces, primero es importante dar una breve explicación sobre cómo se lee/escribe en la memoria.

Supongamos que queremos leer por primera vez un archivo que se encuentra en nuestro disco duro. Lo que hará el Sistema Operativo será buscar en el disco el archivo y guardarlo en memoria además de guardar una copia en un buffer. Esto nos será de gran utilidad ya que si quisiéramos leer el mismo archivo una segunda vez, es mucho más rápido leer del buffer que del disco duro.

Por otro lado, en caso de que quisiéramos escribir información hacia un archivo, al inicio esta información será almacenada solamente en un buffer, ya que como mencionamos anteriormente, las operaciones de lectura y escritura son más rápidas hacia un buffer. Eventualmente, se aplicará un flush y toda la información que se haya escrito hacia el buffer será movida hacia el disco.

Una vez expuesta la introducción, vamos a comprobar por qué resulta más eficiente el uso del memory mapping en la práctica. Llevaremos a cabo un experimento. Inicializamos un primer Array de tamaño 250000 y lo guardaremos en disco. Después, crearemos un segundo Array, accederemos a ciertos espacios para modificar su contenido y lo guardaremos en disco. Determinaremos las mejoras en eficiencia comparando el tiempo de ejecución usando la librería memmap y no usandola.

In [56]:
import mmap
import os.path as path
import numpy as np
from time import process_time
from tempfile import mkdtemp

filename = path.join(mkdtemp(), 'newfile.dat')

# Experimento 1
# Start the counter 
t1_start = process_time()
# Creamos Array de tamaño 90000 
a = np.memmap(filename, mode="write", shape=(500, 500), dtype=np.int)
a.flat = np.arange(500* 500)
# Guardamos el Array en Memoria
a.flush()

b = np.memmap(filename, mode='r+', shape=(500, 500), dtype=np.int)
b[100, :] *= 2
b.flush()

# Stop the counter
t1_stop = process_time()

print(f'Tiempo de ejecución usando la librería memmap ---------> {t1_stop - t1_start}')


# Experimento 2
# Start the counter 
t3_start = process_time()
a_2 = []
for i in range(250000):
    a_2.append(i)

b_2 = []
for i in range(250000):
    if i >= 100000 and i < 100999:
        b_2.append(i*2)
    else:
        b_2.append(i)

# Stop the stopwatch counter
t4_stop = process_time()

print(f'Tiempo de ejecución sin usar la librería memmap -------> {t4_stop - t3_start}')

Tiempo de ejecución usando la librería memmap ---------> 0.015625
Tiempo de ejecución sin usar la librería memmap -------> 0.125


Podemos observar en todas las repeticiones del experimento que el tiempo de ejecución haciendo uso de la librería memmap es menor que cuando no la usamos, concluyendo que es más eficiente manipular partes de un ndarray de gran tamaño con la librería memmap. Uno de los factores que ayuda en la eficiencia es el uso del método flush con el cual podemos indicar cuándo hacer la escritura a disco ahorrando así tiempo. Sin embargo, es importante señalar que estos beneficios se observaron solamente con ndarrays de tamaño muy grande, ya que para pequeños ndarrays la diferencia en tiempo es insignificante cómo para llegar a una conclusión.

## Artículo 02

### Data Structures for Statistical Computing in Python

#### Actividad 02.01 - Desarrollar una opinión razonada del estado actual de las herramientas de análisis de datos estadísticos en contraposición a como se muestran en el artículo, R vs Python vs SQL vs Others...

Haciendo un breve resúmen sobre algunas de las principales herramientas, podemos decir lo siguiente:

- **Python**: lenguaje de programación traducido que es open source y contiene múltiples librerías para el análisis de datos como Pandas y Numpy además de tener múltiples usos como el desarrollo web, de escritorio, visión computacional, etc.
- **R**: lenguaje de programación open source que es utilizado básicamente para el análisis de datos.
- **MatLab**: lenguaje de programación con licencia, utilizando para realizar operaciones numéricas.
- **SQL**: lenguaje de programación utilizado para realizar querys o consultas y recuperar y administrar información de un sistema de bases de datos relacionales.

Aunque estas herramientas no representan la totalidad pues existen otras como Excel, Julia, SAS y muchas más, podríamos decir que son las más populares entre los empleadores y/o personas que se dedican al análisis de datos. El siguiente gráfico nos muestra la popularidad de los diferentes lenguajes de programación en los anuncios de trabajo

![title](programming_languages.png)

Tomando en cuenta las ventajas y desventajas de cada una de las herramientas anteriormente listadas, en mi opinión, Python tiene una ligera ventaja sobre los demás lenguajes de programación, sin embargo, esto no quiere decir que sea mejor, simplemente considero que tiene ciertas ventajas que lo hacen más versátil.

Como mencione anteriormente, considero que Python es el lenguaje más versátil y fácil de aprender, ya que tiene una sintaxis muy simple a comparación de otros como C++ o Java donde no involucra aspectos como el punto y coma, apuntadores, Templates, etc. Por ejemplo:

- **Hello World en Python**\
print("Hello World!")

- **Hello world en C++**\
    using namespace std;\
    #include\<iostream>\
    int main(){
        cout << "Hello World!" << endl;
        return 0;
    }

Cómo podemos ver, para hacer un Hello World, en Python solamente necesitamos de una línea de código, mientras que en C++ ocupamos de más líneas y sintaxis.

Ya que Python es open source, tiene una comunidad muy grande que constantemente brinda apoyo y mantenimiento, por lo que es fácil encontrar foros, documentos o librerías que resuelvan casi cualquier necesidad. Algunos ejemplos de ello son Cython, Pandas, Numpy, MatplotLib, etc. A comparación de Python, MatLab por ejemplo es un lenguaje de programación que no es open source, por lo que sus recursos son más limitados.

Ahora, comparando Python con R, ambos son open source, sin embargo, R es un lenguaje que se encuentra más orientado a personas que quieren realizar análisis de datos y trabajar directamente con modelos matemáticos, ecuaciones y fórmulas. Por otro lado, Python es un lenguaje que ofrece una mayor variedad de opciones para trabajar con los datos y el programador tiene que tomar decisiones como el tipo del objeto que usará, cómo leer el dataset, donde guardar la información, etc.

Es cierto que una de las desventajas de Python comparado con R, es que en Python tenemos que instalar manualmente la mayoría de librerías usadas en el análisis de datos como Pandas y numpy, sin embargo existen soluciones como Anaconda donde se instalan todas estas librerías automáticamente.
    
Cómo mencione anteriormente, considero que Python es el lenguaje de programación más versátil, ya que podemos hacer tanto desarrollo web, como de escritorio, análisis de datos, computer vision, etc. por lo tanto, desde mi punto de vista, Python es el mejor lenguaje para aquellos que deseen aprender a programar y no tengan conocimientos previos, ya que podrán ver conceptos como Programación Orientada a Objetos, Funciones, Recursividad, Estructuras de Datos, Algoritmos, etc. además de tener a la mano poderosas herramientas de análisis de datos como Pandas. 

Si bien R es una herramienta más poderosa para aquellos que trabajan con modelos y análisis matemáticos intensos, creo que librerías como Pandas y Numpy ofrecen cosas muy interesantes que facilitan el trabajo sin dejar de lado la facilidad y versatilidad de Python. 

## Referencias

- https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html
- http://personal.cimat.mx:8181/~mrivera/cursos/python/broadcasting/broadcasting.html
- https://realpython.com/python-mmap/
- https://pythonspeed.com/articles/mmap-vs-zarr-hdf5/
- https://numpy.org/doc/stable/reference/generated/numpy.memmap.html
- http://www.jds-online.com/files/150%E5%AE%8C%E6%88%90V.pdf
- https://towardsdatascience.com/team-r-or-team-python-2f8cf04310e6